In [110]:
using IntervalArithmetic
#interval representations for pi and sqrt(2)
pi_n=pi..pi 
q2=sqrt(2..2)

#fourier transform of phi(x), phi_h
function phi_h(xi)
    return pi_n*exp(-q2*pi_n*abs(xi))*cos(q2*pi_n*abs(xi)-pi_n/4)
end

#B(\xi,p)
function B(xi,p)
    term1=4*p*xi^2*abs(phi_h(xi))/(p^2-2*xi^2)
    term2=2*xi^2*exp(-pi_n*p)*(1/(10*(p-q2*xi))+15/(100*(p+q2*xi)^2))
    return term1+term2
end

#H(\xi,n)
function H(xi,n)
    val=pi_n*(-1)^n*exp(-pi_n*abs(n)) #phi_h(n/sqrt(2))
    val=val/q2
    res=2*xi^2*(phi_h(xi)-val)
    return res/((n-q2*xi)^2)
end

lim=-(pi_n/q2)*tanh(pi_n/2) #\sum_{n \in \mathbb{Z}}phi_h(n/sqrt(2)).
#want to show R_low>=lim

eps=2^(-6)


function H_tilde(xi,n)
    diff=abs(xi-n/q2)
    if (diff>=eps || n<0)
        return H(xi,n)
    end
    secterm=diff*((xi^2/6)*exp(pi_n*q2*diff)*8*pi_n^4*exp(-pi_n*q2*xi)+2*pi^3*exp(-pi_n*n)*(xi+n/q2))
    return -(n^2/2)*q2*pi_n^3*((-1)^n)*exp(-pi_n*n)-secterm
end
          
#lower bound for \mathcal{R}(\xi)
function R_low(xi,p)
    inter=phi_h(xi)-pi_n/q2 #phi_h(\xi)-phi_h(0)
    for n=1:p
        inter=inter+H_tilde(xi,n)+H_tilde(xi,-n)
    end
    return inter-B(xi,p)
end



function check(lower,upper,p)
    beg=lower
    print(beg, "\n")
    last=9.0
    temp=beg..last #iteratively checking R_low(xi,p)>=lim on interval tem[beg,last]
    while beg<upper+1/1024
        res=R_low(temp,p) #range of values of R_low([beg,last])
        iters=0
        while !(((res ∩ lim)==∅) & (res>=lim)) #iteratively decrease last until R_low([temp])>=last
            last=(beg+last)/2
            temp=beg..last
            res=R_low(temp,p)
            iters=iters+1
            if iters%10==0
                print("On iteration ",iters, "\n")
            end
            if iters>=50
                last=nextfloat(beg)
                iters=0
            end
        end
        beg=last
        last=beg+1
        temp=beg..last
        print("Checked up to ",beg, "\n")  
    end
    print("Checked on interval ", "[",lower,",",upper,"]","\n")
end


        

check (generic function with 1 method)

In [111]:
check(0,9,10000)

0
Checked up to 0.28125
Checked up to 0.40625
Checked up to 0.46875
Checked up to 0.53125
Checked up to 0.5625
Checked up to 0.59375
Checked up to 0.609375
Checked up to 0.625
Checked up to 0.6328125
Checked up to 0.640625
Checked up to 0.6484375
Checked up to 0.65234375
Checked up to 0.65625
Checked up to 0.66015625
Checked up to 0.6640625
Checked up to 0.666015625
Checked up to 0.66796875
Checked up to 0.669921875
Checked up to 0.671875
Checked up to 0.673828125
Checked up to 0.67578125
On iteration 10
Checked up to 0.6767578125
On iteration 10
Checked up to 0.677734375
On iteration 10
Checked up to 0.6787109375
On iteration 10
Checked up to 0.6796875
On iteration 10
Checked up to 0.6806640625
On iteration 10
Checked up to 0.681640625
On iteration 10
Checked up to 0.6826171875
On iteration 10
Checked up to 0.68359375
On iteration 10
Checked up to 0.6845703125
On iteration 10
Checked up to 0.685546875
On iteration 10
Checked up to 0.68603515625
On iteration 10
Checked up to 0.68652343

In [97]:
function tau1_term(n)
    return (142*pi_n*n^2*exp(-pi_n*n))/(9*q2)
end

function tau2_term(n)
    return n^2*(8*n^2+6*(1+2*n)^2)*pi_n*exp(-pi_n*n)/q2
end

function tail_tau1(b)
    return (142*pi_n*exp(-pi_n*b/3))/(9*q2*(1-exp(-pi_n/3)))
end

function tail_tau2(m)
    return (62*pi_n*exp(-pi_n*m/5))/(q2*(1-exp(-pi_n/5)))
end
    

b=100
m=100
val=tail_tau1(b)+tail_tau2(m)

for n=2:b-1
    val=val+tau1_term(n)
end

for n=5:m-1
    val=val+tau2_term(n)
end
    
val


[0.297849, 0.29785]

In [105]:
(6*8^2-1)/((q2*8+1)^2)

[2.52592, 2.52593]

In [101]:
function tau_1(b)
    res=0
    for n=1:b-1
        res=res+3*q2*pi_n*n^2*((-1)^(n-1))*exp(-pi_n*n)
        if n>=2
            res=res-8*n^3*(pi_n*exp(-pi_n*n)/q2)
        end
    end
    return res-(7*q2*pi_n*exp(-2*pi_n*b/5))/(1-exp(-2*pi_n/5))
end

b=1000

tau_1(b)-((2*pi_n^2*exp(0.5*pi_n)+12*q2*exp(0.5*pi_n)+20)*3^2+q2*exp(0.5*pi_n))*pi_n*exp(-3*q2*pi_n)
    


[0.167776, 0.167778]

In [102]:
2*pi_n^3*exp(pi_n/2)

[298.309, 298.311]

In [103]:
q2*pi_n*exp(pi_n/2)

[21.3723, 21.3724]

In [104]:
(5/2)*q2*pi_n*exp(-pi_n)-31/100-2*(396*8^4+25*8^2)*exp(-q2*pi_n*8)

[0.169985, 0.169986]

In [106]:
pi_n/(q2*(exp(pi_n)-1))

[0.100332, 0.100334]

In [107]:
pi_n/(q2*exp(pi_n))

[0.0959971, 0.0959972]